In [1]:
import folium
import pandas as pd

grant_export = 'P3_GrantExport.csv'
data  = pd.read_csv(grant_export, sep=';', error_bad_lines=False)

print(data.shape)

(63969, 15)


In [2]:
L = []
for st in data['University'] :
    a=str(st).split(" - ")
    if len(a) > 1 :
        L.append( str(st).split(" - ")[1])
    else :
        L.append('NaN')
data['Grant Money'] = L

In [3]:
count_university = data['Grant Money'].value_counts()

In [4]:
cantons_data = pd.DataFrame(data=count_university)
cantons_data.head(100)


Grant Money
NaN              12982
ZH                6774
GE                6394
ETHZ              6153
BE                5473
BS                4746
EPFL              4428
LA                4092
NA                2595
FR                2079
NE                1596
NPO               1473
PSI                538
FP                 492
SG                 426
USI                346
EAWAG              333
HES-SO             271
ZFH                260
EMPA               238
FHNW               228
WSL                223
LU                 214
IHEID              198
BFH                136
SUPSI              135
AGS                135
IDIAP               88
FMI                 84
ASPIT               81
...                ...
FIBL                17
HEPL                16
PHLU                14
IRO                 14
PHSG                13
PHBern              13
EHB                 12
IKG                 12
SPF                 10
CREALP               8
BITG                 8
PHZG                 7
HEPFR                7
HEPBEJUNE            7
PHGR                 7
PHTG                 6
FUS                  6
ISR                  6
RWS                  6
PHSZ                 5
IRSOL                5
FHKD                 3
STHB                 3
ISSKA                3
SHLR                 2
FTL                  2
PHVS                 2
ASP                  2
PHSH                 1
FFHS                 1

[77 rows x 1 columns]

In [29]:
# Import data from GeoName
import requests
import json

def getCanton(place):
    base_url = 'http://api.geonames.org/postalCodeLookupJSON'
    payload = {'username': 'adaisp',
               'country' : 'CH',
               'placename' : place
                }
    response = requests.get(base_url, params=payload)
    data = json.loads(response.text)
    
    return data['postalcodes'][0]['adminName1']

# Example of using this function !
print(getCanton('Ecublens'))

Canton de Vaud


In [18]:
uni_data = data.loc[:, ['University', 'Approved Amount']]

## /!\ Doesn't work yet... we should transform the data type of the column 'approved amount' to integer !
## in order to be able to sum everything and not "concatenate" strings
uni_data = uni_data.groupby(['University'])['Approved Amount'].sum()
uni_data.head()

University
AO Research Institute - AORI                          367063.0072271.00100000.00651159.00105353.0033...
Allergie- und Asthmaforschung - SIAF                  1161184.001665894.00114308.001791129.003000.00...
Berner Fachhochschule - BFH                           6000.00364680.00330678.0015000.0034675.0017500...
Biotechnologie Institut Thurgau - BITG                197000.00382961.00376152.00209256.00205110.003...
Centre de rech. sur l'environnement alpin - CREALP    292924.00227126.00145918.00290000.00199980.002...
Name: Approved Amount, dtype: object

In [28]:
# Creating the map !
cantons_geo = 'ch-cantons.topojson.json'

map = folium.Map(location=[46.73, 8.2], zoom_start=4)

map.choropleth(geo_path=cantons_geo, data=cantons_data,
             columns=['University', 'Approved Amount'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Approved Amount')

/Users/Sym/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


TypeError: can't multiply sequence by non-int of type 'float'